In [ ]:
## Garbages
import wrds
db=wrds.Connection(wrds_username='debabech', wrds_username='Electro1004$')
db.create_pgpass_file()
permcos = db.get_table(library='crsp', table='stocknames')[["permco","ticker"]]
ticker = asset
permco = permcos[permcos.ticker == ticker].permco.values[0]

req = f"select prc, date from crsp.dsf where permco in ({permco}) and date >='2012-01-31' and date <='2021-02-16'"
asset = db.raw_sql(req, date_cols=['date'])

asset = asset.dropna()


In [ ]:
class FastTextSentiment(Base):
    """Predict fine-grained sentiment scores using FastText"""
    def __init__(self, model_file: str=None) -> None:
        super().__init__()
        import fasttext
        self.model = fasttext.load_model(model_file)

    def score(self, text: str) -> int:
        # Predict just the top label (hence 1 index below)
        labels, probabilities = self.model.predict(text, 1)
        pred = int(labels[0][-1])
        return pred

    def predict(self, train_file: None, test_file: str, lower_case: bool) -> pd.DataFrame:
        df = self.read_data(test_file, lower_case)
        df['pred'] = df['text'].apply(self.score)
        return df

In [ ]:
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap

PRE_TRAINED_MODEL_NAME = 'bert-base-cased' # since “BAD” might convey more sentiment than “bad” on a forum

tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

tokens = tokenizer.tokenize(XXX)
token_ids = tokenizer.convert_tokens_to_ids(tokens)

encoding = tokenizer.encode_plus(
    XXX,
  max_length=32,
  add_special_tokens=True, # Add '[CLS]' and '[SEP]'
  return_token_type_ids=False,
  pad_to_max_length=True,
  return_attention_mask=True,
  return_tensors='pt',  # Return PyTorch tensors)

encoding.keys()

dict_keys(['input_ids', 'attention_mask'])    

token_lens = []

for txt in df.content:
  tokens = tokenizer.encode(txt, max_length=512)
  token_lens.append(len(tokens))
  
sns.distplot(token_lens)
plt.xlim([0, 256])
plt.xlabel('Token count')

MAX_LEN = XXX #See plot to select sequence size, take a bit higher

class GPReviewDataset(Dataset):

  def __init__(self, reviews, targets, tokenizer, max_len):
    self.reviews = reviews
    self.targets = targets
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
    return len(self.reviews)

  def __getitem__(self, item):
    review = str(self.reviews[item])
    target = self.targets[item]
    
    encoding = self.tokenizer.encode_plus(
      review,
      add_special_tokens=True,
      max_length=self.max_len,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt',
    )

    return {
      'review_text': review,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      'targets': torch.tensor(target, dtype=torch.long)
    }
    
df_train, df_test = train_test_split(
  df,
  test_size=0.1,
  random_state=RANDOM_SEED
)

df_val, df_test = train_test_split(
  df_test,
  test_size=0.5,
  random_state=RANDOM_SEED
)